In [2]:
import pygeostat as pg
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [6]:
data = pd.read_csv("red13.dat", delimiter=";")
data = data.set_index("Hole ID")
data

,Northing,Elevation,"Thickness, m",Au [g/t],Ag [g/t],"Cu, percent","Zn, percent",Rock Type
Hole ID,,,,,,,,
67,20341.5,-36.0,0.34,2.16,6.17,2.82,2.2,20
68,20344.0,-66.0,5.04,2.88,9.94,7.39,2.3,20
69,20303.0,-47.0,0.29,0.00,2.40,0.22,0.0,20
71,20305.0,-96.0,0.45,0.62,4.11,0.70,0.0,20
72,20260.0,-100.0,1.19,1.65,6.86,2.57,0.1,20
...,...,...,...,...,...,...,...,...
133,20176.0,-163.0,5.01,2.38,16.76,5.31,1.0,20
134,20281.0,-235.0,1.06,0.29,2.67,1.39,4.0,20
136,20227.0,-157.0,1.96,1.35,2.06,1.11,0.1,20


In [ ]:
data.scatter(x=["Elevation"], y)